### Here, I explore qubit and cavity drives

##### There's something very strange I don't really understand. In Krastanov's SNAP paper, he defined 
$$ \hat{H}_1 = \epsilon(t) e^{i \omega_c t} \hat{a}^\dag + H.c. $$

##### Why is there a $e^{i \omega_c t}$ factor? Heisenberg picture??

##### I find that without this $e^{i \omega_c t}$ factor, then $\alpha = i \int \epsilon(t) dt$ does indeed recover $\hat{D}(\alpha)$.

##### I'm gonna check with Tanjung about it. But for now, I think I'll remove the factor first.

In [16]:
import numpy as np
from scipy.linalg import expm
import scipy
import matplotlib.pyplot as plt
import time

In [17]:
np.set_printoptions(suppress=True)
np.set_printoptions(precision=3)

In [18]:
cavity_dim = 8
omega_q = 21
omega_c = 20
Delta = omega_q - omega_c
chi = 1 / Delta

T = 1
dt = 0.01
t_axis = np.arange(0, T, dt)[0:]

In [19]:
sigmaI = np.array(
    [
        [1, 0],
        [0, 1],
    ]
)
sigmaX = np.array(
    [
        [0, 1],
        [1, 0],
    ]
)
# sigmaZ = np.array(
#     [
#         [-1, 0],
#         [0, 1],
#     ]
# )
sigmaBin = np.array(
    [
        [0, 0],
        [0, 1],
    ]
)

# Annihilation operator
a = np.zeros((cavity_dim, cavity_dim))
n_indices = np.arange(1, cavity_dim)
a[n_indices - 1, n_indices] = np.sqrt(n_indices)
a_dag = a.conjugate().transpose()
# print(a)
# print(a_dag)

def Fock(n):
    vec = [[0] for _ in range(cavity_dim)]
    vec[n] = [1]
    return np.array(vec)
# print(Fock(1))

In [20]:
drive_qubit = np.array([0 for t in t_axis])
drive_cavity = np.array([1/T for t in t_axis])

def H(i):
    # from the SNAP paper, alpha = 1j * integrate(drive_cavity * dt)
    return (
        # omega_q * np.kron(sigmaBin, np.identity(cavity_dim))
    #   + (omega_c - chi*np.kron(sigmaBin, np.identity(cavity_dim))) * (np.kron(sigmaI, a_dag) @ np.kron(sigmaI, a))
    #   + drive_cavity[i] * np.exp(1j * omega_c * i*dt) * np.kron(sigmaI, a_dag) + drive_cavity[i].conjugate() * np.exp(-1j * omega_c * i*dt) * np.kron(sigmaI, a)
      + drive_cavity[i] * np.kron(sigmaI, a_dag) + drive_cavity[i].conjugate() * np.kron(sigmaI, a)
    #   + drive_qubit[i] * np.exp(1j * omega_q * i*dt) * np.kron(np.array([[0,0],[1,0]]), np.identity(cavity_dim)) + drive_qubit[i].conjugate() * np.exp(-1j * omega_q * i*dt) * np.kron(np.array([[0,1],[0,0]]), np.identity(cavity_dim))
    )

def D(alpha):
    return expm(alpha * np.kron(sigmaI, a_dag) - np.conjugate(alpha) * np.kron(sigmaI, a))
# def D(alpha):
#     return expm(alpha * a_dag - np.conjugate(alpha) * a)

def CoheState(alpha):
    ket = np.zeros((cavity_dim, 1), dtype=np.complex128)
    ket[0] = 1. + 0j
    for n in range(1, cavity_dim):
        ket[n] = ket[n-1] * alpha / np.sqrt(n)
    return ket / np.linalg.norm(ket)
    # return ket * expm(-np.abs(alpha)**2 / 2)
    

In [21]:
1j * sum(drive_cavity * dt)

1.0000000000000007j

In [22]:
def H(i):
    # from the SNAP paper, alpha = 1j * integrate(drive_cavity * dt)
    return (
        # omega_q * np.kron(sigmaBin, np.identity(cavity_dim))
    #   + (omega_c - chi*np.kron(sigmaBin, np.identity(cavity_dim))) * (np.kron(sigmaI, a_dag) @ np.kron(sigmaI, a))
    #   + drive_cavity[i] * np.exp(1j * omega_c * i*dt) * np.kron(sigmaI, a_dag) + drive_cavity[i].conjugate() * np.exp(-1j * omega_c * i*dt) * np.kron(sigmaI, a)
      + drive_cavity[i] * np.kron(sigmaI, a_dag) + drive_cavity[i].conjugate() * np.kron(sigmaI, a)
    #   + drive_qubit[i] * np.exp(1j * omega_q * i*dt) * np.kron(np.array([[0,0],[1,0]]), np.identity(cavity_dim)) + drive_qubit[i].conjugate() * np.exp(-1j * omega_q * i*dt) * np.kron(np.array([[0,1],[0,0]]), np.identity(cavity_dim))
    )

# H(len(t_axis)-1)

Unitary = np.identity(len(H(0)))
for i in range(len(t_axis)):
    Unitary = expm(-1j * H(i) * dt) @ Unitary
# print(Unitary)

In [23]:
Unitary @ np.kron( np.array([[1], [0]]), Fock(0))

array([[ 0.607+0.j   ],
       [ 0.   -0.607j],
       [-0.429+0.j   ],
       [ 0.   +0.248j],
       [ 0.124+0.j   ],
       [ 0.   -0.055j],
       [-0.022+0.j   ],
       [ 0.   +0.01j ],
       [ 0.   +0.j   ],
       [ 0.   +0.j   ],
       [ 0.   +0.j   ],
       [ 0.   +0.j   ],
       [ 0.   +0.j   ],
       [ 0.   +0.j   ],
       [ 0.   +0.j   ],
       [ 0.   +0.j   ]])

In [24]:
D(1j) @ np.kron( np.array([[1], [0]]), Fock(0))

array([[ 0.607+0.j   ],
       [ 0.   +0.607j],
       [-0.429+0.j   ],
       [ 0.   -0.248j],
       [ 0.124+0.j   ],
       [ 0.   +0.055j],
       [-0.022+0.j   ],
       [ 0.   -0.01j ],
       [ 0.   +0.j   ],
       [ 0.   +0.j   ],
       [ 0.   +0.j   ],
       [ 0.   +0.j   ],
       [ 0.   +0.j   ],
       [ 0.   +0.j   ],
       [ 0.   +0.j   ],
       [ 0.   +0.j   ]])

In [25]:
np.kron( np.array([[1], [0]]), CoheState(1j))

array([[ 0.607+0.j   ],
       [ 0.   +0.607j],
       [-0.429+0.j   ],
       [ 0.   -0.248j],
       [ 0.124+0.j   ],
       [ 0.   +0.055j],
       [-0.023+0.j   ],
       [ 0.   -0.009j],
       [ 0.   +0.j   ],
       [ 0.   +0.j   ],
       [-0.   +0.j   ],
       [ 0.   -0.j   ],
       [ 0.   +0.j   ],
       [ 0.   +0.j   ],
       [-0.   +0.j   ],
       [ 0.   -0.j   ]])

In [26]:
def H(i):
    # from the SNAP paper, alpha = 1j * integrate(drive_cavity * dt)
    return (
        # omega_q * np.kron(sigmaBin, np.identity(cavity_dim))
    #   + (omega_c - chi*np.kron(sigmaBin, np.identity(cavity_dim))) * (np.kron(sigmaI, a_dag) @ np.kron(sigmaI, a))
      + drive_cavity[i] * np.exp(1j * omega_c * i*dt) * np.kron(sigmaI, a_dag) + drive_cavity[i].conjugate() * np.exp(-1j * omega_c * i*dt) * np.kron(sigmaI, a)
    #   + drive_cavity[i] * np.kron(sigmaI, a_dag) + drive_cavity[i].conjugate() * np.kron(sigmaI, a)
    #   + drive_qubit[i] * np.exp(1j * omega_q * i*dt) * np.kron(np.array([[0,0],[1,0]]), np.identity(cavity_dim)) + drive_qubit[i].conjugate() * np.exp(-1j * omega_q * i*dt) * np.kron(np.array([[0,1],[0,0]]), np.identity(cavity_dim))
    )

# H(len(t_axis)-1)

Unitary = np.identity(len(H(0)))
for i in range(len(t_axis)):
    Unitary = expm(-1j * H(i) * dt) @ Unitary
# print(Unitary)

In [27]:
Unitary @ np.kron( np.array([[1], [0]]), Fock(0))

array([[ 0.997+0.047j],
       [ 0.027-0.047j],
       [-0.001-0.002j],
       [-0.   +0.j   ],
       [-0.   +0.j   ],
       [ 0.   +0.j   ],
       [ 0.   -0.j   ],
       [ 0.   -0.j   ],
       [ 0.   +0.j   ],
       [ 0.   +0.j   ],
       [ 0.   +0.j   ],
       [ 0.   +0.j   ],
       [ 0.   +0.j   ],
       [ 0.   +0.j   ],
       [ 0.   +0.j   ],
       [ 0.   +0.j   ]])

In [28]:
D(1j) @ np.kron( np.array([[1], [0]]), Fock(0))

array([[ 0.607+0.j   ],
       [ 0.   +0.607j],
       [-0.429+0.j   ],
       [ 0.   -0.248j],
       [ 0.124+0.j   ],
       [ 0.   +0.055j],
       [-0.022+0.j   ],
       [ 0.   -0.01j ],
       [ 0.   +0.j   ],
       [ 0.   +0.j   ],
       [ 0.   +0.j   ],
       [ 0.   +0.j   ],
       [ 0.   +0.j   ],
       [ 0.   +0.j   ],
       [ 0.   +0.j   ],
       [ 0.   +0.j   ]])

In [29]:
np.kron( np.array([[1], [0]]), CoheState(1j))

array([[ 0.607+0.j   ],
       [ 0.   +0.607j],
       [-0.429+0.j   ],
       [ 0.   -0.248j],
       [ 0.124+0.j   ],
       [ 0.   +0.055j],
       [-0.023+0.j   ],
       [ 0.   -0.009j],
       [ 0.   +0.j   ],
       [ 0.   +0.j   ],
       [-0.   +0.j   ],
       [ 0.   -0.j   ],
       [ 0.   +0.j   ],
       [ 0.   +0.j   ],
       [-0.   +0.j   ],
       [ 0.   -0.j   ]])